In [ ]:
import translators as ts
from os import path
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import numpy as np
import pandas as pd
from transformers import pipeline
import plotly.express as px

ts._google.language_map

In [ ]:
import snscrape.modules.twitter as sntwitter

# Creating list to append tweet data to
tweets_list2 = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('geocode:1.2494,103.8303,2km lang:id since:2019-01-01 until:2020-01-01').get_items()):
    if i>5000:
        break
    tweets_list2.append([tweet.date, tweet.id, tweet.content, tweet.user.username])

# sample search string
# sentosa (#sentosa) lang:id since:2022-01-01 until:2022-03-31'
# https://github.com/igorbrigadir/twitter-advanced-search
# https://developer.twitter.com/en/docs/twitter-api/v1/data-dictionary/object-model/geo#place
# with the tweet id you can punch it into a browser to view the tweet post twitter.com/anyuser/status/{tweet ID}

# Creating a dataframe from the tweets list above
tweets_df2 = pd.DataFrame(tweets_list2, columns=['Datetime', 'Tweet Id', 'Text', 'Username'])

In [ ]:
removed_tzone=tweets_df2
# remove time zonne as excel cannot handle time zone
removed_tzone['Datetime'] = removed_tzone['Datetime'].dt.tz_localize(None)
tweetid= removed_tzone['Tweet Id']

removed_tzone

In [ ]:
removed_tzone.to_excel('tweets.xlsx', sheet_name='sheet1', index=False)
# saving results to excel as emojis are unable to be saved in csv form
tweetid.to_csv('tweetsID.csv', index=False)
# due to the tweet ID being too long. it has be to saved separately

In [ ]:
# code for reading excel
df=pd.read_excel('geo location 2019 tweets wo emoji translated.xlsx')  

df

In [ ]:
text_col=df

text_col

In [ ]:
text_col['english_text'] = text_col['Text'].apply(lambda x: ts.google(x, from_language='id', to_language='en'))

text_col

In [ ]:
# for name, values in removed_tzone["Text"].iteritems():
#     print(values)
import demoji
# test_tweet = removed_tzone["Text"]
# test_tweet
n=0
for v in removed_tzone["Text"]:
    raw_tweet = str(v)
    tweet_removed_emoji = demoji.replace_with_desc(raw_tweet)
    removed_tzone.loc[n,"Text"] = tweet_removed_emoji
    n+=1
    

In [ ]:
tweetid= removed_tzone['Tweet Id']

removed_tzone.to_excel('tweets wo emoji.xlsx', sheet_name='sheet1', index=False)
# tweetid.to_csv('tweetsID wo emoji.csv', index=False)

In [ ]:
t_df = pd.DataFrame(text_col, columns = ['english_text'])

t_df

In [ ]:
text = t_df["english_text"].to_string(index=False)
text_raw = text_col["Text"].to_string(index=False)
print ("There are {} words in the combination of all tweets.".format(len(text)))
print ("There are {} words in the combination of all tweets.".format(len(text_raw)))

In [ ]:
wordcloud = WordCloud(width=1600, height=800).generate(text)

plt.figure(figsize=[20,10])
plt.imshow(wordcloud,interpolation='bilinear')
plt.axis("off")

plt.show()

In [ ]:
wordcloud_raw = WordCloud(width=1600, height=800).generate(text_raw)

plt.figure(figsize=[20,10])
plt.imshow(wordcloud_raw,interpolation='bilinear')
plt.axis("off")

plt.show()

In [ ]:
word_count = WordCloud().process_text(text)

word_count

In [ ]:
word_count_raw = WordCloud().process_text(text_raw)

word_count_raw

In [ ]:
pretrained_name = "sahri/indonesiasentiment"

nlp = pipeline("sentiment-analysis",
              model=pretrained_name,
              tokenizer=pretrained_name)

In [ ]:
def sentiment_analysis_label(text):
    '''
    Sentiment analysis labelling
    0 - positive
    1 - negative
    '''
    output = nlp(text)
    if output[0]['label'] == 'positive':
        label = 0
    elif output[0]['label'] == 'neutral':
        label = 1
    else:
        label = 2
    return label

In [ ]:
def sentiment_analysis_score(text):
    '''
    Return sentiment analysis score
    '''
    output = nlp(text)
    return round(output[0]['score'], 3)

In [ ]:
df

In [ ]:
df['sentiment'] = df['Text'].apply(sentiment_analysis_label)

In [ ]:
df['sentiment'].value_counts(ascending=False)

In [ ]:
df['score'] = df['Text'].apply(sentiment_analysis_score)

In [ ]:
df.to_excel('geo location 2019 tweets wo emoji translated senti.xlsx', sheet_name='sheet1', index=False)

In [ ]:
df=pd.read_excel('geo location 2019 tweets wo emoji translated senti.xlsx')  

df

In [ ]:
px.histogram(df, x="Datetime", y="sentiment")